In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences

## 1. Load the Pokemon Dataset

In [29]:
df = pd.read_csv('Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [30]:
df.shape

(800, 13)

In [31]:
#column names
df.columns.values

array(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'], dtype=object)

## 2. Set up features and labels for neural network

In [32]:
# It seems that we won't need the following columns of data:
# '#', 'Type 2', 'Generation', 'Legendary', so we can drop them
df_concise = df.drop(['#', 'Type 2', 'Generation', 'Legendary'], axis=1)
df_concise.head()

,Name,Type 1,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,Bulbasaur,Grass,318,45,49,49,65,65,45
1,Ivysaur,Grass,405,60,62,63,80,80,60
2,Venusaur,Grass,525,80,82,83,100,100,80
3,VenusaurMega Venusaur,Grass,625,80,100,123,122,120,80
4,Charmander,Fire,309,39,52,43,60,50,65


Because there are only 800 different pokemons with 800 unique values for "Name", it seems to me less appealing to use "Name" as an input in classifying the neural network. Therefore, I will try in this section to set up the features by (Type 1, Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed) and the labels as Type 1.

In [33]:
# Find the unique Type-1 types among all pokemons
unique_types = np.unique(df["Type 1"].as_matrix())
print(unique_types)

# Create type2index dictionary to store indices for each type
type2index = {}
for i, ty in enumerate(unique_types):
    type2index[ty] = i

# Check type2index
type2index

['Bug' 'Dark' 'Dragon' 'Electric' 'Fairy' 'Fighting' 'Fire' 'Flying'
 'Ghost' 'Grass' 'Ground' 'Ice' 'Normal' 'Poison' 'Psychic' 'Rock' 'Steel'
 'Water']


{'Bug': 0,
 'Dark': 1,
 'Dragon': 2,
 'Electric': 3,
 'Fairy': 4,
 'Fighting': 5,
 'Fire': 6,
 'Flying': 7,
 'Ghost': 8,
 'Grass': 9,
 'Ground': 10,
 'Ice': 11,
 'Normal': 12,
 'Poison': 13,
 'Psychic': 14,
 'Rock': 15,
 'Steel': 16,
 'Water': 17}

Then, we can extract all the relevant data from our df_concise dataframe.

In [34]:
# Selecting using df.loc and check that the relevant columns are what we need
df_concise.loc[:, ['Type 1', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].head()

,Type 1,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,Grass,318,45,49,49,65,65,45
1,Grass,405,60,62,63,80,80,60
2,Grass,525,80,82,83,100,100,80
3,Grass,625,80,100,123,122,120,80
4,Fire,309,39,52,43,60,50,65


In [35]:
# Extract the related columns of df_concise and convert into numpy arrays
data = df_concise.loc[:, ['Type 1', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]\
    .as_matrix()

# Check data
data

array([['Grass', 318, 45, ..., 65, 65, 45],
       ['Grass', 405, 60, ..., 80, 80, 60],
       ['Grass', 525, 80, ..., 100, 100, 80],
       ..., 
       ['Psychic', 600, 80, ..., 150, 130, 70],
       ['Psychic', 680, 80, ..., 170, 130, 80],
       ['Fire', 600, 80, ..., 130, 90, 70]], dtype=object)

Now, let's shuffle the data randomly so that we can start setting up training and testing features and labels

In [36]:
np.random.shuffle(data)
data

array([['Ice', 580, 80, ..., 100, 200, 50],
       ['Dark', 330, 45, ..., 80, 50, 65],
       ['Electric', 580, 90, ..., 125, 90, 100],
       ..., 
       ['Poison', 467, 100, ..., 73, 83, 55],
       ['Bug', 450, 70, ..., 90, 75, 90],
       ['Psychic', 278, 38, ..., 65, 55, 50]], dtype=object)

In [37]:
# Set up the features and labels
labels = data[:, 0]
features = data[:, 1:]
print(labels.shape, features.shape)

(800,) (800, 7)


Now is time to categorize the labels according to our type2index

In [38]:
# Switch word strings for each label into corresponding index from type2index dictionary
for i in range(len(labels)):
    labels[i] = type2index[labels[i]]

# Categorize the labels using to_categorical from tflearn
labels = to_categorical(labels, nb_classes=unique_types.shape[0])
print(unique_types.shape[0])

18


In [39]:
# Now we have labels categorized successfully
labels[:10]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,

Now is time to separate the training and testing sets. Since there are 800 samples in total, I will choose to take 700 training sets and 100 testing sets.

In [40]:
# Separate the training and testing sets; X means features, Y means labels
trainX, testX = features[:700], features[700:]
trainY, testY = labels[:700], labels[700:]

print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(700, 7) (700, 18)
(100, 7) (100, 18)


## 3. Setting up TFLearn Neural Network

In [41]:
tf.reset_default_graph()

# Set up input layer
net = tflearn.input_data([None, 7])

# Set up hidden layers with ReLU
net = tflearn.fully_connected(net, 200, activation="ReLU")
net = tflearn.fully_connected(net, 100, activation="ReLU")
net = tflearn.fully_connected(net, 250, activation="ReLU")
net = tflearn.fully_connected(net, 250, activation="ReLU")

# Set up output layer with softmax
net = tflearn.fully_connected(net, 18, activation="softmax")
net = tflearn.regression(net, optimizer="sgd", learning_rate = 0.001, loss="categorical_crossentropy")

model = tflearn.DNN(net)
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=10)

Training Step: 49  | total loss: 2.88885 | time: 0.024s
| SGD | epoch: 010 | loss: 2.88885 - acc: 0.1303 -- iter: 512/630
Training Step: 50  | total loss: 2.88888 | time: 1.038s
| SGD | epoch: 010 | loss: 2.88888 - acc: 0.1343 | val_loss: 2.88908 - val_acc: 0.1000 -- iter: 630/630
--


In [42]:
# Compare the labels that our model predicts with the actual labels
predictions = (np.array(model.predict(testX))[:,0] >= 0.5).astype(np.int_)
# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
test_accuracy = np.mean(predictions == testY[:,0], axis=0)
# Print out the result
print("Test accuracy: ", test_accuracy)

Test accuracy:  0.88
